In [ ]:
# https://youtu.be/TCH_1BHY58I
# https://github.com/karpathy/makemore 

In [ ]:
# we now go to MLP (multilayer perceptron)....(using NLP (natural language processing))
# 'a neural probabilistic language model' (2003) chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
# embedding of words and transfer knowledge....comment in class....
# fig 1: 4th word predicted after the three....
import random
import torch
import torch.nn.functional as  F
import matplotlib.pyplot as plt #for making figures
%matplotlib inline

In [ ]:
# read in all the words
random.seed(158)
words=open('Data/nomi_italiani.txt','r').read().splitlines()
random.shuffle(words)
words[0:10]

In [ ]:
len(words)

In [ ]:
# build the vocabulary of characters and mapping to/from integers
chars=sorted(list(set(''.join(words))))

stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos ={i:s for s,i in stoi.items()}
print(itos)
print(stoi)

In [ ]:
# build the dataset

block_size =3 #context length: how many characters do we take to predict the next one ... change it !!
# try: block_size=1 ...Markov Chain, then try = 2 and =10
X,Y = [],[]  # input & label

for w in words[0:5]:
    print(w)
    context=[0]*block_size
    for ch in w +'.':
        ix=stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context=context[1:]+[ix]  # shift: crop and append
X=torch.tensor(X)
Y=torch.tensor(Y)

In [ ]:
Y

In [ ]:
X.shape, X.dtype,Y.shape,Y.dtype

In [ ]:
X

In [ ]:
Y

In [ ]:
# we want a model that input X and predict Y ......
# first  the embedding in d= 2 of the 28 th character ( in the paper d=30 with a 17k vocabulary...)

In [ ]:
C=torch.randn((28,2))  #https://pytorch.org/docs/stable/generated/torch.randn.html

In [ ]:
C[5] # equivalent to one-hot encording....!! index= neuron input...first layer...comment in class

In [ ]:
C.shape

In [ ]:
#visualizing the embedding.....

plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data,s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(),itos[i], ha="center",va="center", color="white")
    plt.grid('minor')
    
    


In [ ]:
F.one_hot(torch.tensor(5),num_classes=28)  #comment this in class.....

In [ ]:
F.one_hot(torch.tensor(5),num_classes=28).float()@C   #comment this in class.....

In [ ]:
# i.e. the first (linear) layer (fig. in the paper).....

In [ ]:
X.shape

In [ ]:
# How we embed the 41 3-ngram's ??
emb=C[X] # the beauty of tensor....
emb.shape

In [ ]:
# moreover C is also "differentiable" !!

In [ ]:
X[14]

In [ ]:
X[14,2]

In [ ]:
C[X]

In [ ]:
C[X][14,:]

In [ ]:
C[X][14,2]

In [ ]:
X[14]

In [ ]:
C[15,:]

In [ ]:
emb.shape

In [ ]:
# construct the Layer.... x.W+ b ... so the input has dimension 6=3*2 for (say) 100 neurons...
W1=torch.randn(6,100)
b1=torch.randn(100)


In [ ]:
# 'roughly' we want to compute emb @ W1 + b1....we need to 'concatenate'....discuss in class..
emb @ W1 + b1

In [ ]:
# https://pytorch.org/docs/stable/torch.html search for concatenate...
# we want emb[:,0,:],emb[:,1,:],emb[:,2,:]  show in class.....

torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],1)
#torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],1).shape


In [ ]:
# we want a code for general n-grams..... 
# use 'unbind' https://pytorch.org/docs/stable/generated/torch.unbind.html#torch.unbind
len(torch.unbind(emb,1))


In [ ]:
emb

In [ ]:
torch.unbind(emb,1)

In [ ]:
# and this work fore any context length.......

torch.cat(torch.unbind(emb,1),1)

In [ ]:
# there is a better way....

# https://pytorch.org/docs/stable/generated/torch.Tensor.view.html

# https://pytorch.org/docs/stable/generated/torch.Tensor.stride.html

# use google image and discuss in class....

a=torch.arange(18)
a

In [ ]:
a.shape


In [ ]:
a.view(9,2)

In [ ]:
a.view(2,9)

In [ ]:
a.view(3,2,3)

In [ ]:
# very efficient in torch....discuss in class
a.storage()

In [ ]:
emb.view(41,6) == torch.cat(torch.unbind(emb,1),1)

In [ ]:
# so we can use

h=emb.view(41,6) @ W1 + b1

In [ ]:
h

In [ ]:
h.shape

In [ ]:
# more general...
h=emb.view(-1,6) @ W1 + b1

In [ ]:
# first layer

# https://pytorch.org/docs/stable/generated/torch.tanh.html

h=torch.tanh(emb.view(-1,6) @ W1 + b1)

In [ ]:
h

In [ ]:
 #second layer...think about dimensions....
    
W2=torch.randn((100,28))
b2=torch.randn(28)

In [ ]:
logits= h @ W2 +b2
logits.shape

In [ ]:
counts = logits.exp()

In [ ]:
prob=counts/counts.sum(1,keepdims=True)

In [ ]:
prob[0]

In [ ]:
prob[0].sum()

In [ ]:
# we now the 'true'' forth caracher....
Y

In [ ]:
prob[0]

In [ ]:
prob[1]

In [ ]:
prob[[0,1]]
#prob[0,1]

In [ ]:
prob[[0,1],[2,5]]

In [ ]:
torch.arange(41)

In [ ]:
prob[torch.arange(41),Y]  #comments the results...random weights.....


In [ ]:
loss= - prob[torch.arange(41),Y].log().mean()
loss  # very bad of course.....

In [ ]:
# ------- now we made it respectable .......


In [ ]:
X.shape, Y.shape #data set

In [ ]:
g=torch.Generator().manual_seed(123456780)# for reproducibility
C=torch.randn((28,2), generator=g)
W1=torch.randn((6,100), generator=g)
b1=torch.randn(100, generator=g)
W2=torch.randn((100,28), generator=g)
b2=torch.randn(28, generator=g)
parameters=[C,W1,b1,W2,b2]

In [ ]:
sum(p.nelement() for p in parameters)# number of parameter in total...

In [ ]:
emb=C[X] # torch.Size([41, 3, 2])
h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(41,100)
logits= h @ W2 + b2 #(41,27)
counts=logits.exp()
prob=counts/counts.sum(1,keepdims=True)
loss=-prob[torch.arange(41),Y].log().mean()
loss

In [ ]:
# we can make it more efficient using the build in cross entropy loss function...
F.cross_entropy(logits,Y)

In [ ]:
# so..... https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
emb=C[X] # torch.Size([41, 3, 2])
h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(41,100)
logits= h @ W2 + b2 #(41,27)
#counts=logits.exp()
#prob=counts/counts.sum(1,keepdims=True)
#loss=-prob[torch.arange(41),Y].log().mean()
loss=F.cross_entropy(logits,Y)
loss

In [ ]:
# two very good reasons to use 'cross_entropy': more efficient (no tensor) and substract the maximum to avoid nan....discuss....

logits=torch.tensor([-5,-3,0,10]) # -100
counts=logits.exp()
prob=counts/counts.sum()
print(counts)
prob

In [ ]:
for p in parameters:
    p.requires_grad = True

for _ in range(1000):
    # now we learn...forward bass 
    emb=C[X] # torch.Size([41, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(41,100)
    logits= h @ W2 + b2 #(41,27)
    loss=F.cross_entropy(logits,Y)
#    print(loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1*p.grad

In [ ]:
# sampling from the model.....

g=torch.Generator().manual_seed(12345678+10)

for _ in range(20):
    out=[]
    context=[0]*block_size
    while True:
        emb=C[torch.tensor([context])]
        h=torch.tanh(emb.view(1,-1) @ W1 + b1) 
        logits= h @ W2 + b2
        probs=F.softmax(logits,dim=1)
        ix=torch.multinomial(probs,num_samples=1,generator=g).item()
        context=context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
        
    print(''.join(itos[i] for i in out))
        

In [ ]:
# with 'only' 41 examples and 3584 parameters we are overfitting !!


for p in parameters:
    p.requires_grad = True

for _ in range(1000):
    # now we learn...forward bass 
    emb=C[X] # torch.Size([41, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(41,100)
    logits= h @ W2 + b2 #(41,27)
    loss=F.cross_entropy(logits,Y)
    #print(loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1*p.grad
print(loss.item())

In [ ]:
logits.max(1)

In [ ]:
Y # 'almost' perfect fitting....discuss in class

In [ ]:
# build the dataset...using NOW all words...

block_size =3 #context length: how many characters do we take to predict the next one ... change it !!
X,Y = [],[]  # input & label

#for w in words[0:5]:    before....
for w in words:
  #  print(w)
    context=[0]*block_size
    for ch in w +'.':
        ix=stoi[ch]
        X.append(context)
        Y.append(ix)
      #  print(''.join(itos[i] for i in context), '--->', itos[ix])
        context=context[1:]+[ix]  # shift: crop and append
X=torch.tensor(X)
Y=torch.tensor(Y)

In [ ]:
X.shape, Y.shape

In [ ]:
# exactly as before....
g=torch.Generator().manual_seed(123456780)# for reproducibility
C=torch.randn((28,2), generator=g)
W1=torch.randn((6,100), generator=g)
b1=torch.randn(100, generator=g)
W2=torch.randn((100,28), generator=g)
b2=torch.randn(28, generator=g)
parameters=[C,W1,b1,W2,b2]

In [ ]:

for p in parameters:
    p.requires_grad = True

In [ ]:


for _ in range(10):
    # now we learn...forward bass -- = 73643
    emb=C[X] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Y)
    print(loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1*p.grad
print(loss.item())

In [ ]:
# see how it is 'slowing down'...we always update on all the examples...we are going to use mini batches as usual...

ix=torch.randint(0,X.shape[0],(10,)) #try ix=torch.randint(0,X.shape[0],(10,2)) and explain
#https://pytorch.org/docs/stable/generated/torch.randint.html
ix

In [ ]:
ix=torch.randint(0,X.shape[0],(10,1))
ix

In [ ]:
for _ in range(10):
    
    # mini batch construct of size ...
    ix=torch.randint(0,X.shape[0],(32,))
    # now we learn...forward bass -- = 73643
    emb=C[X[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Y[ix])
    print(loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1*p.grad
print(loss.item())

In [ ]:
# how we define the 'learning rate' ? p.data += -0.1*p.grad
# play with learning rate from .01 to 100.... and discuss



In [ ]:
for _ in range(50):
    
    # mini batch construct
    ix=torch.randint(0,X.shape[0],(100,))
    # now we learn...forward bass -- = 73643
    emb=C[X[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Y[ix])
    print(loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    for p in parameters:
        p.data += -.1*p.grad
#print(loss.item())

In [ ]:
lre=torch.linspace(-3,0,1000)
lrs= 10**lre
lrs

In [ ]:
for p in parameters:
    p.requires_grad = True
    
lri=[]
lriex=[]
lossi=[]
    

for i in range(1000):
    
    # mini batch construct
    ix=torch.randint(0,X.shape[0],(100,))
    # now we learn...forward bass -- = 73643
    emb=C[X[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Y[ix])
    print(i,loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    lr=lrs[i]
    # lr= .01
    for p in parameters:
        p.data += -lr*p.grad
#print(loss.item())

#track stats
    lri.append(lr)
    lriex.append(lre[i])
    lossi.append(loss.item())

In [ ]:
plt.plot(lri,lossi)  # discuss......

In [ ]:
plt.plot(lriex,lossi)  # discuss......

In [ ]:
# we can do already much better than bigram....almost...remember overfitting.... 

emb=C[X] # torch.Size([41, 3, 2])
h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(41,100)
logits= h @ W2 + b2 #(41,27)
loss=F.cross_entropy(logits,Y)
loss

In [ ]:
# training split, dev/validation split (for hypermarameters !!), test split
# 80%, 10%, 10%
# be careful with the test eugene.....

def build_dataset(words):
    block_size =3 #context length: how many characters do we take to predict the next one ... change it !!
    X,Y = [],[]  # input & label

   
    for w in words:
        context=[0]*block_size
        for ch in w +'.':
            ix=stoi[ch]
            X.append(context)
            Y.append(ix)
      #  print(''.join(itos[i] for i in context), '--->', itos[ix])
            context=context[1:]+[ix]  # shift: crop and append
    X=torch.tensor(X)
    Y=torch.tensor(Y)
    print(X.shape,Y.shape)
    return X,Y



    

In [ ]:
import random
random.seed(42)
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))

Xtr,Ytr = build_dataset(words[:n1])
Xdev,Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])



In [ ]:
# and we do it again with the new datasets.......
Xtr.shape, Ytr.shape

# exactly as before....
g=torch.Generator().manual_seed(123456780)# for reproducibility
C=torch.randn((28,2), generator=g)
W1=torch.randn((6,100), generator=g)
b1=torch.randn(100, generator=g)
W2=torch.randn((100,28), generator=g)
b2=torch.randn(28, generator=g)
parameters=[C,W1,b1,W2,b2]

In [ ]:
for p in parameters:
    p.requires_grad = True
    
    
lre=torch.linspace(-3,0,1000)
lrs= 10**lre


In [ ]:
# now we train only on Xtr

lri=[]
lriex=[]
lossi=[]
    

for i in range(10000):
    
    # mini batch construct
    ix=torch.randint(0,Xtr.shape[0],(40,))
    # now we learn...forward bass -- = 73643
    emb=C[Xtr[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Ytr[ix])
   # print(i,loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    #lr=lrs[i]
    lr= .1
    for p in parameters:
        p.data += -lr*p.grad
print(loss.item())

#track stats
#    lri.append(lr)
#    lriex.append(lre[i])
#    lossi.append(loss.item())

In [ ]:
# now we evaluate on Xdev
emb=C[Xdev]
h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
logits= h @ W2 + b2 #(--,27)
loss=F.cross_entropy(logits,Ydev)
loss

In [ ]:
# now we evaluate on Xtr..... we are NOT overfitting
emb=C[Xtr]
h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
logits= h @ W2 + b2 #(--,27)
loss=F.cross_entropy(logits,Ytr)
loss

In [ ]:
# now we cahnge the hyper parameters....
# before....
#g=torch.Generator().manual_seed(123456780)# for reproducibility
#C=torch.randn((28,2), generator=g)
#W1=torch.randn((6,100), generator=g)
#b1=torch.randn(100, generator=g)
#W2=torch.randn((100,28), generator=g)
#b2=torch.randn(28, generator=g)
#parameters=[C,W1,b1,W2,b2]

g=torch.Generator().manual_seed(123456780)# for reproducibility
C=torch.randn((28,2), generator=g)
W1=torch.randn((6,300), generator=g)
b1=torch.randn(300, generator=g)
W2=torch.randn((300,28), generator=g)
b2=torch.randn(28, generator=g)
parameters=[C,W1,b1,W2,b2]

In [ ]:
sum(p.nelement() for p in parameters)# number of parameter in total... before 3584

In [ ]:
lri=[]
lriex=[]
lossi=[]
stepi=[]
for p in parameters:
    p.requires_grad = True

for i in range(10000):
    
    # mini batch construct
    ix=torch.randint(0,Xtr.shape[0],(40,))
    # now we learn...forward bass -- = 73643
    emb=C[Xtr[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,6) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Ytr[ix])
   # print(i,loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    #lr=lrs[i]
    lr= .1
    for p in parameters:
        p.data += -lr*p.grad
    stepi.append(i)
    lossi.append(loss.item())

In [ ]:
plt.plot(stepi,lossi)

In [ ]:
# Exercise with different hypermarameters and learning rate.....describe and discuss results....

In [ ]:
#visualizing the embedding.....

plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(),itos[i], ha="center",va="center", color="white")
    plt.grid('minor')

In [ ]:
# we scale up the embedding dimension and see what happen


g=torch.Generator().manual_seed(123456780)# for reproducibility
C=torch.randn((28,10), generator=g)
W1=torch.randn((30,200), generator=g)
b1=torch.randn(200, generator=g)
W2=torch.randn((200,28), generator=g)
b2=torch.randn(28, generator=g)
parameters=[C,W1,b1,W2,b2]

In [ ]:
sum(p.nelement() for p in parameters)# number of parameter in total

In [ ]:
lri=[]
lriex=[]
lossi=[]
stepi=[]

for p in parameters:
    p.requires_grad = True

In [ ]:
# do lr =0.1 and then 0.01 and look at the performance....


for i in range(200000):
    
    # mini batch construct
    ix=torch.randint(0,Xtr.shape[0],(40,))
    # now we learn...forward bass -- = 73643
    emb=C[Xtr[ix]] # torch.Size([--, 3, 2])
    h=torch.tanh(emb.view(-1,30) @ W1 + b1) #(--,100)
    logits= h @ W2 + b2 #(--,27)
    loss=F.cross_entropy(logits,Ytr[ix])
   # print(i,loss.item())
    # backward pass 
    for p in parameters:
        p.grad=None
    loss.backward()
    #update
    #lr=lrs[i]
    lr= .1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr*p.grad
    stepi.append(i)
    lossi.append(loss.log10().item()) # note the log10 !!

In [ ]:
plt.plot(stepi,lossi)

In [ ]:
emb=C[Xtr]
h=torch.tanh(emb.view(-1,30) @ W1 + b1) #(--,100)   30 not 6 !
logits= h @ W2 + b2 #(--,27)
loss=F.cross_entropy(logits,Ytr)
loss

In [ ]:
emb=C[Xdev]
h=torch.tanh(emb.view(-1,30) @ W1 + b1) #(--,100)
logits= h @ W2 + b2 #(--,27)
loss=F.cross_entropy(logits,Ydev)
loss

In [ ]:
# several (hyper)parameters to paly with: number of layer, numbers of neurons fro layers, embedding dimensions..dimension of the batches, learning rate....
# now you can read the cited paper on language models......


In [ ]:
# sampling from the model.....

g=torch.Generator().manual_seed(12345678+10)

for _ in range(30):
    out=[]
    context=[0]*block_size
    while True:
        emb=C[torch.tensor([context])]
        h=torch.tanh(emb.view(1,-1) @ W1 + b1) 
        logits= h @ W2 + b2
        probs=F.softmax(logits,dim=1)
        ix=torch.multinomial(probs,num_samples=1,generator=g).item()
        context=context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
        
    print(''.join(itos[i] for i in out))
        